In [1]:
#coding:utf-8
import pandas as pd
import time

from sklearn.metrics import log_loss
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
from scipy.stats import norm, skew #加载一些统计的库
import gc

In [3]:
print('train')
train = pd.read_table('train.txt',sep=' ',index_col=False)

train


In [5]:
def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']
    
    print('context_timestamp_ing')
    data['context_timestamp'] = data['context_timestamp'].apply(time2cov)
    
    return data

In [ ]:
train = pre_process(train)

In [4]:
train.columns

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'is_trade'],
      dtype='object')

In [7]:
train[['instance_id','item_id']].groupby('item_id').count()

,instance_id
item_id,
42172734419742,543
119705681289325,2
139232509090699,76
186228696000288,3
208192717221960,5
271501162750665,323
334417922587028,2
345352588519597,134
412546248320450,1


In [14]:
train = train[['instance_id','item_id']]

In [8]:
print('test')
test = pd.read_table('test.txt',sep=' ',index_col=False)

test


In [15]:
test = test[['instance_id','item_id']]

In [21]:
all_data = train.append(test)

In [22]:
all_data

,instance_id,item_id
0,7118167540371,7595487976415452415
1,43508800019098,5340996313837249587
2,78067858668749,8363638779510498577
3,137525720365644,7121266959776152715
4,151193932520734,5738516817201102842
5,191034067765499,3165167092384011466
6,221786632908010,2157590763338091652
7,233375766178961,7296864533820220362
8,235542268576836,3948722777398180782
9,271028956022181,4158769038163595899


In [23]:
all_data = all_data.drop_duplicates(subset='item_id')

In [24]:
all_data[['instance_id','item_id']].groupby('item_id').count()

,instance_id
item_id,
29268323426272,1
42172734419742,1
119705681289325,1
139232509090699,1
186228696000288,1
208192717221960,1
271501162750665,1
334417922587028,1
345352588519597,1


In [14]:
test[['instance_id','user_id']].groupby('user_id').count()

,instance_id
user_id,
46479832058031,1
95722635043429,1
142686228656106,2
182607826951001,1
185036739105852,2
193762749479175,1
238761307739117,1
260779748630644,4
308545487928585,1


In [8]:
del(train)
gc.collect()

2832

In [11]:
print('train')
train = pd.read_table('day_7.txt',sep=' ',index_col=False)

train


In [12]:
train[['instance_id','item_id']].groupby('item_id').count()

,instance_id
item_id,
42172734419742,83
139232509090699,13
271501162750665,14
345352588519597,20
412546248320450,1
584480686684468,3
1189938696856115,6
1410182414862995,55
1629778605881953,12
